# Chữ Nôm recognition, nhưng nó lạ lắm

Based on [Kuzushiji Recognition just like Digit Recognition](https://www.kaggle.com/basu369victor/kuzushiji-recognition-just-like-digit-recognition/notebook)

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from os import listdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, BatchNormalization,Input
from keras.models import Model,load_model
from IPython.display import SVG
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
#%matplotlib inline

In [ ]:
fontsize = 50

font = ImageFont.truetype('NotoSansCJKjp-Regular.otf', fontsize, encoding = 'utf-8')

In [ ]:
df_train = pd.read_csv('train.csv')
unicode_map = {codepoint: char for codepoint, char in pd.read_csv('unicode_translate.csv').values}

In [ ]:
df_train.isnull().sum()

In [ ]:
# Return the image containing the bounding boxes and characters annotated
def visualise_training_data(image_fn, labels):
    labels = np.array(labels.split(' '))[:-1].reshape(-1, 5)
    
    imsource = Image.open(image_fn).convert('RGBA')
    bbox_canvas = Image.new('RGBA', imsource.size)
    char_canvas = Image.new('RGBA', imsource.size)
    bbox_draw = ImageDraw.Draw(bbox_canvas)
    char_draw = ImageDraw.Draw(char_canvas)
    
    for label, x1, y1, x2, y2 in labels:
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        h, w = abs(x1 - x2), abs(y1 - y2)
        
        char = unicode_map[label]
        
        bbox_draw.rectangle((x1, y1, x2, y2), fill = (255, 255, 255, 0), outline = (255, 0, 0, 255))
        char_draw.text((x1 + w + fontsize / 4, y1 + h/2 - fontsize), char, fill = (0, 0, 255, 255), font = font)
        
        cropped_image = imsource.crop((x1, y1, x2, y2))
        plt.figure()
        print(char)
        plt.imshow(cropped_image)
        plt.show()
    
    imsource = Image.alpha_composite(Image.alpha_composite(imsource, bbox_canvas), char_canvas)
    imsource = imsource.convert('RGB')
    
    return np.asarray(imsource)

In [ ]:
np.random.seed(1337)

for i in range(1):
    img, labels = df_train.values[np.random.randint(len(df_train))]
    visualise = visualise_training_data('data_real/img/{}'.format(img), labels)
    
    plt.figure(figsize = (15, 15))
    plt.title(img)
    plt.imshow(visualise, interpolation = 'lanczos')
    plt.show()

In [ ]:
def preprocessImage(image):
    ret, th1 = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)

In [ ]:
def ExtractData():
    X_, Y_ = [], []
    
    for img, labels in tqdm(df_train[:420].values):
        try:
            image_fn = 'data_real/img/{}'.format(img)
            labels = np.array(labels.split(' '))[:-1].reshape(-1, 5)
    
            imsource = Image.open(image_fn).convert('RGBA')
            bbox_canvas = Image.new('RGBA', imsource.size)
            char_canvas = Image.new('RGBA', imsource.size)
            bbox_draw = ImageDraw.Draw(bbox_canvas)
            char_draw = ImageDraw.Draw(char_canvas)

            for label, x1, y1, x2, y2 in labels:
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                h, w = abs(x1 - x2), abs(y1 - y2)

                char = unicode_map[label]

                cropped_image = imsource.crop((x1, y1, x2, y2))
                image = cropped_image.resize((300, 300))
                image = np.asarray(image)
                image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                ret, th1 = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY_INV)

                X_.append(th1)
                Y_.append(str(char))
        except:
            pass
        
    X_ = np.array(X_)
    Y_ = np.array(Y_)

In [ ]:
XX_, yy_ = ExtractData()

plt.figure()
plt.imshow(XX_[99])